In [61]:
import sys
sys.path.append("C:\\Users\\KlimovaAnnaA\\Documents\\MyFiles\\Projects\\Working_attributes")
from Imports import * 

In [62]:
Today = '2024-09-30'

print_Debt = True
print_Deriv = False
excel_tofolder_on_Z = True

In [63]:
quantum_columns = str(export_from_RISKCUSTOM("""select *from "RISKACCESS"."quantumDebt" fetch first 1 rows only""").columns.tolist()).replace('\'','\"')[1:-1]

C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\Working_attributes\Defs.py:92: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_export = pd.read_sql(query, con=connection)


In [64]:
# quantum debt
query = f"""
with table_1 as
(select 
{quantum_columns},
--"reportDate","effectiveFrom","entity","dealNo","holding","counterparty","amount","eventCurrency","amountUSD","actionDate","dealClass1","dealClass2","instrumentOwner","dealSet","facility", 
MAX("effectiveFrom") OVER (partition BY "reportDate") as "max_effectiveFrom"
from "RISKACCESS"."quantumDebt" 
where "reportDate" = TO_DATE('{Today}', 'YYYY-MM-DD')
--fetch first 10 rows only
)
select * from table_1
where "effectiveFrom" = table_1."max_effectiveFrom"
--fetch first 10 rows only
"""
data_Debt_export = export_from_RISKCUSTOM(query)
data_Debt_export.shape

(7492, 74)

Фильтрация данных

In [96]:
assert len(data_Debt_export[data_Debt_export.entity.isna()]) == 0 # data quality
data_Debt_work = data_Debt_export.copy() # copy data
# merge
data_Debt_work['entity_code'] = merge_Mapping(data_Debt_work, col='entity')
data_Debt_work.loc[data_Debt_work.entity_code=='External', 'entity_code'] = data_Debt_work.loc[data_Debt_work.entity_code=='External', 'entity']
data_Debt_work['Entity_group'] = merge_SalesUnits(data_Debt_work, col='entity_code', merge_col='ocpSegment')

# Выполнение цикла для рассчета amount, amountUSD и присваивания значения counterparty для каждого DealSet

# Выбор нужного разреза данных
deals_data_Bonds = data_Debt_work.loc[data_Debt_work.dealSet.str.contains('Bond'), ['entity_code', 'Entity_group', 'counterparty','amount', 'amountUSD', 'instrumentOwner', 'dealSet','reportDate']] # Фильтрация разреза по значениям в DealSet, которые содержат слово Bond

Bonds_list = deals_data_Bonds.dealSet.unique()

deals_data_Bonds_work = pd.DataFrame()
for Bond in Bonds_list:
    try:
        Bond_data = deals_data_Bonds[deals_data_Bonds.dealSet==Bond] # Отбор данных одного Bond
        Bond_data_traded = Bond_data[Bond_data.instrumentOwner=='EC_TRADED_BONDS']
        Bond_data_issued = Bond_data[Bond_data.instrumentOwner=='EC_ISSUED_BONDS']
        if len(Bond_data_issued)>1:
            Bond_data_issued['index_copy'] = Bond_data_issued.index # copy index
            # groupby
            Bond_data_issued_pivot = Bond_data_issued.groupby(as_index=False, dropna=False, by=['entity_code',	'Entity_group',	'counterparty', 'instrumentOwner',	'dealSet',	'reportDate']).agg({'amount':'sum', 'amountUSD':'sum', 'index_copy':'min'})
            Bond_data_issued_pivot.index = [int(Bond_data_issued_pivot.index_copy.item())] # index on min in group
            Bond_data_issued_pivot.drop(columns='index_copy') # drop index copy col
            Bond_data = pd.concat([Bond_data_traded, Bond_data_issued_pivot])
        assert len(Bond_data_issued) != 0

        columns_list = ['amount', 'amountUSD']
        for column in columns_list:
            Traded_data_sum = float(Bond_data.loc[Bond_data.instrumentOwner=='EC_TRADED_BONDS', column].sum())
            Issued_data = float(Bond_data.loc[Bond_data.instrumentOwner=='EC_ISSUED_BONDS', column].item())
            # Внесение нового значения amount в issued bonds
            Bond_data.loc[Bond_data['instrumentOwner'] =="EC_ISSUED_BONDS", column] = Traded_data_sum + Issued_data 
        # Внесение нового значения counterparty в traded bonds
        Issied_data_entity = Bond_data.loc[Bond_data.instrumentOwner=='EC_ISSUED_BONDS', 'entity_code'].squeeze()
        Bond_data.loc[Bond_data['instrumentOwner'] =="EC_TRADED_BONDS", 'counterparty'] = Issied_data_entity
        
        deals_data_Bonds_work = pd.concat([deals_data_Bonds_work, Bond_data])
        # deals_data_Bonds[deals_data_Bonds.dealSet==Bond] = Bond_data
    except:
        continue

index_list = deals_data_Bonds_work.index.to_list()
data_Debt_work.loc[(data_Debt_work.dealSet.str.contains('Bond'))&(data_Debt_work.index.isin(index_list)), ['entity_code', 'Entity_group', 'counterparty','amount', 'amountUSD', 'instrumentOwner', 'dealSet','reportDate']] = deals_data_Bonds_work
data_Debt_work.loc[(data_Debt_work.dealSet.str.contains('Bond'))&(~data_Debt_work.index.isin(index_list)), 'entity_code'] = np.nan
data_Debt_work = data_Debt_work[~data_Debt_work.entity_code.isna()]
# Запись выполненного цикла в в основные данные

data_Debt_work = Period(data_Debt_work, day_for_count='reportDate', col_with_date='actionDate', day_is_col=True).reset_index(drop=True)

data_Debt_work.loc[data_Debt_work.counterparty.str.contains('(EC)'), 'counterparty_name'] = data_Debt_work.loc[data_Debt_work.counterparty.str.contains('(EC)'), 'counterparty'].str[:-5] 
# Обрезка " (EC)" в каждой строке

assert len(data_Debt_export[data_Debt_export.counterparty.isna()])==0
data_Debt_work['cty_Code'] = merge_Mapping(data_Debt_work, col='counterparty_name')
data_Debt_work['Counterparty_Group'] = merge_SalesUnits(data_Debt_work, col='cty_Code', merge_col='ocpSegment')
# Merge counterparty с Mapping и SalesUnits

data_Debt_work['Source'] = 'Quantum'

C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\Working_attributes\Defs.py:92: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_export = pd.read_sql(query, con=connection)
C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\Working_attributes\Defs.py:33: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['SPV' 'SPV' 'SGC' ... 'SGC' 'SGC' 'SGC']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  merge_data.loc[merge_data[f'{id_colmn}_merge'] != 'External', 'last_merge'] = merge_data.loc[merge_data[f'{id_colmn}_merge'] != 'External', f'{id_colmn}_merge']
C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_10360\1178440806.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFram

Запись Debt в Excel:

In [98]:
if print_Debt == True:
    for holding in data_Debt_work.holding.unique().tolist():
        Output_Debt = "_".join([str(date.today()), holding ,'quantum_Debt.xlsx'])
        if excel_tofolder_on_Z == True:
            Output_Debt = Output_path + Output_Debt
        data_Debt_print = data_Debt_work.loc[data_Debt_work.holding==holding, ['entity_code', 'Entity_group','counterparty',\
                                    'Counterparty_Group','amount','eventCurrency','amountUSD','actionDate','Days',\
                                    'Period','dealClass1','dealClass2','instrumentOwner','dealSet','facility','Source']]\
                                    .rename(columns={'amount': 'amoutn outstanding',\
                                                        'eventCurrency': 'Currency',\
                                                        'amountUSD': 'amount USD eq',\
                                                        'actionDate': 'termEnd'})
        data_Debt_print.to_excel(Output_Debt, sheet_name = 'Debt', index = False)

In [67]:
check = data_Debt_work[['entity_code', 'Entity_group', 'counterparty','amount', 'amountUSD', 'instrumentOwner', 'dealSet']]
check = check[check.instrumentOwner.str.contains('BONDS')].reset_index(drop=True)
check[check.dealSet == check.dealSet.unique().tolist()[3]]

,entity_code,Entity_group,counterparty,amount,amountUSD,instrumentOwner,dealSet
16,RUEMO,RUFERT,NSD RU (EC),-1.000000e+10,-1.078602e+08,EC_ISSUED_BONDS,MCC RuBond 07 2020-04 R10b


In [68]:
manual_map = data_Debt_work.loc[(data_Debt_work['cpartyType'] == 'Internal') & (data_Debt_work['entity_code'] == 'External'), ['counterparty', 'entity_code', 'Counterparty_Group']].drop_duplicates()
manual_map

manual_map2 = data_Debt_work.loc[data_Debt_work['Entity_group'] == 'External', ['entity_code', 'Entity_group']].drop_duplicates()
manual_map2

manual_map_print = pd.concat([manual_map, manual_map2], axis=1)
manual_map_print
# manual_map_print.to_excel('Manual_map.xlsx', index=False)

,counterparty,entity_code,Counterparty_Group,entity_code,Entity_group


Работа с Forwards и Swaps:

In [69]:
query = f"""select * 
from "RISKACCESS"."quantumForwards" 
where "positionDate" = TO_DATE('{Today}', 'YYYY-MM-DD')"""
data_Forwards_export = export_from_RISKCUSTOM(query)

query = f"""select * 
from "RISKACCESS"."quantumSwaps" 
where "positionDate" = TO_DATE('{Today}', 'YYYY-MM-DD')"""
data_Swaps_export = export_from_RISKCUSTOM(query)

data_Deriv_qu = pd.concat([data_Forwards_export,data_Swaps_export])
data_Deriv_qu.head(2)

C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\Working_attributes\Defs.py:92: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_export = pd.read_sql(query, con=connection)
C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_10360\1716991512.py:11: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  data_Deriv_qu = pd.concat([data_Forwards_export,data_Swaps_export])


,##batch_id,##deleted,##inserted,##origin_name,adjustedProfitLoss,baseCurrency,baseType,checkSettlementDateAlt,contractRate,counterparty,...,recMargin,recPrincipalFreq,recRateBasisCode,recRateBasisFreq,recRateBasisName,recRateFormula,recRatesetFreq,shortInterestRate,shortNPVOfAdjustedProfitLoss,startdate
0,41150557@WHPROD,N,2024-10-17 01:00:19,QUANTUM,2079.82,USD,CONTRACT,Y,1.365100,Western Alliance US (EC),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
1,41150557@WHPROD,N,2024-10-17 01:00:19,QUANTUM,1883.60,USD,CONTRACT,Y,1.367691,Western Alliance US (EC),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT


In [70]:
# проверка вхождения в debt
Deriv_dealNo = data_Deriv_qu.dealNo.unique().tolist()
Debt_dealNo = data_Debt_export.dealNo.unique().tolist()
Deriv_dealNo_new = [x for x in Deriv_dealNo if x not in Debt_dealNo]
data_Deriv_qu_work = data_Deriv_qu[data_Deriv_qu.dealNo.isin(Deriv_dealNo_new)]
# merge
data_Deriv_qu_work['Entity_group'] = merge_SalesUnits(data_Deriv_qu_work, col='entity', merge_col='ocpSegment')
# melt data
melt_data = pd.melt(data_Deriv_qu_work, id_vars=['positionDate','holdingEntity','Entity_group','entity','counterparty','dealNo','payFaceValue','recFaceValue','maturityDate'], value_vars=['payFXCurrency','recFXCurrency'], value_name='Currency').sort_values('dealNo').rename(columns={'recFaceValue':'amount'})
melt_data.loc[melt_data.variable=='payFXCurrency', 'amount'] = melt_data.loc[melt_data.variable=='payFXCurrency', 'payFaceValue'] * -1
melt_data = melt_data.drop(columns=['payFaceValue','variable']).reset_index(drop=True).sort_values('dealNo')
# Создание Days и Period
melt_data = Period(melt_data, day_for_count=Today, col_with_date='maturityDate')
# создание payFaceValue_in_USD
melt_data = add_in_currency_column(melt_data, CCY_to='USD', col_with_CCY='Currency', date_is_column=False, col_with_VAL='amount', DATE=Today)
# Создание пустых столбцов и столбца ресурса
new_columns = ['Company', 'Settlement', 'Notional_Amount_(USD)', 'Unnamed', 'source']
melt_data = melt_data.reindex(columns=(melt_data.columns.tolist() + new_columns))
melt_data['source'] = 'Quantum'

C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\Working_attributes\Defs.py:92: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_export = pd.read_sql(query, con=connection)
C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\Working_attributes\Defs.py:33: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['NAMSALES' 'NAMSALES' 'NAMSALES' 'NAMSALES' 'NAMSALES' 'NAMSALES'
 'NAMSALES' 'NAMSALES' 'NAMSALES' 'NAMSALES' 'NAMSALES' 'NAMSALES'
 'NAMSALES' 'NAMSALES' 'NAMSALES' 'NAMSALES' 'NAMSALES']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  merge_data.loc[merge_data[f'{id_colmn}_merge'] != 'External', 'last_merge'] = merge_data.loc[merge_data[f'{id_colmn}_merge'] != 'External', f'{id_colmn}_merge']
C:\Users\Klimov

Запись Deriv в файл:

In [71]:
if print_Deriv == True:
    for holding in melt_data.holdingEntity.unique().tolist():
        Output_Deriv = "_".join([str(date.today()), holding ,'quantum_Deriv.xlsx'])
        if excel_tofolder_on_Z == True:
            Output_Deriv = Output_path + Output_Deriv
        
        data_Deriv_print = melt_data.loc[melt_data.holdingEntity==holding,['entity','Entity_group','holdingEntity','Company',\
        'counterparty','dealNo','Currency','amount',\
        'amount_in_USD','Settlement','Notional_Amount_(USD)',\
        'Unnamed','maturityDate','Days','Period','source']]
        data_Deriv_print.to_excel(Output_Deriv, sheet_name='Deriv', index=False)